# Tutorial 1 - Analyzing scRNA-seq data at the Protein Activity Level

This Tutorial will guide the user thorugh the analysis of single-cell RNA-seq (scRNA-seq) data at the Protein Activity level using `pyviper`. Specifically, we will show how to transform a gene expression signature generated from sparse scRNA-seq data into a protein activity matrix using the [VIPER](https://www.nature.com/articles/ng.3593) algorithm. After a brief description of the installation procedure and the modules needed, this Jupyter Notebook is organized in the following sections:\

[**Step 1**. Load a gene expression "signature" at the single cell-level](#1)\
[**Step 2**. Load a gene regulatory network inferred with ARACNe](#2)\
[**Step 3**. Analyze single-cells at the Protein Activity level](#3)\
[**Step 4**. Key Takeaways](#4)

### Install Pyviper
Install `pyviper` from PyPI using pip. Alternatively, refer to the README in the current GitHub to install from the local directory.

In [ ]:
# !pip install pyviper

### Import modules

In [1]:
import pyther
import scanpy as sc
import anndata 
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore", message=".*The 'nopython' keyword.*") # for jit decorator issue with sc.pp.neighbors (09/30/2023)

<a id='1'></a>
### Step 1. Load a gene expression "signature" at the single-cell level
Load gene expression signature to be used as input to the `viper` function for Protein Activity inference. Store the gene expression signature into an [AnnData](https://anndata.readthedocs.io/en/latest/) object to enable interoperability with [scanpy](https://scanpy-tutorials.readthedocs.io/en/latest/#). Display matrix dimensions with cells on rows and features (genes) on columns (after transposition). The gene expression signature was generated from a population of malignant ductal cells from publicly avaiable data from [Peng et al., 2019](https://www.nature.com/articles/s41422-019-0195-y). Please, refer to Tutorial 2 for additional details on how to generate a gene expression signature.  

In [ ]:
gene_expr_url = "https://zenodo.org/records/10056139/files/Tutorial_1_gExpr_fibroblast_5802.tsv.gz"

gene_expr_signature = pd.read_csv(gene_expr_url, sep="\t") # read from remote 
gene_expr_signature = sc.AnnData(gene_expr_signature) # convert to AnnData object

gene_expr_signature

<a id="2"></a>
## Step 2. Load a gene regulatory network inferred with ARACNe 
Load and inspect lineage-specific gene regulatory network generated with the ARACNe3. See [ARACNe3](https://www.mdpi.com/1099-4300/25/3/542) and [ARACNe-AP](https://pubmed.ncbi.nlm.nih.gov/27153652/) for additional information on current ARACNe implementations. 

In [ ]:
network_path = "https://zenodo.org/records/10056139/files/fibroblast-net.tsv" # path to ARACNe network
    
network = pd.read_csv(network_path, delimiter="\t")

Other algorithms can be used to generate gene regulatory networks, but we recommend postprocessing the output to the suitable dataframe format, with columns displayed above.

Convert the pandas DataFrame to an object of class `Interactome` to enable easier manipulation and show the number of regulons using the `size()` method. Type `help(pyther.Interactome)` to see all available methods.

In [ ]:
network_interactome = pyther.Interactome('fibroblasts', network) # convert to class Interactome
network_interactome.size() # show number of regulons in the Interactome

Display the first 5 interactions in the regulatory network. `mor` and `likelihood` represent the mode of regulation of the given regulator-target pair and the likelihood of the interaction, respectively.

In [ ]:
network_interactome.net_table.head()

Filter out targets in the `Interactome` that are not present in the gene expression matrix. This step is recommended before running `viper`. 

In [ ]:
network_interactome.filter_targets(gene_expr_signature.var_names)

As an example, display the number of targets of a couple of regulators, MYC and SERPINA12.

In [ ]:
n_MYC = len(network_interactome.get_reg('MYC')) # number of MYC targets in the network
n_EGFR = len(network_interactome.get_reg('EGFR')) # number of SERPINA12 targets in the network

print("The number of targets of MYC and SERPINA12 are " + str(n_MYC) + " and " + str(n_EGFR) + ", respectively.")

<a id='3'></a>

### Step 3. Convert the gene expression signature into a protein activity matrix using viper
`viper` transforms a gene expression signature into a protein activity matrix by performing enrichment analysis on regulons. Two methods for enrichment analysis are allowed: [aREA](https://www.nature.com/articles/ng.3593) (default) and [NaRnEA](https://www.mdpi.com/1099-4300/25/3/542) (by setting `enrichment="narnea"`).

#### Method 1 - protein activity inference using aREA
As we have seen (section above), different regulators can potentially have different number of targets. We prune each regulon to have the same number of targets (50 in this case). This step is advisable to avoid regulators with an exceedingly number of targets to dominate those with fewer when computing the NES, thus making NES scores comparable.

In [ ]:
network_pruned = network_interactome.copy() # generate a copy of the unpruned network before pruning it
network_pruned.prune(max_targets=50,eliminate=True) # prune interactome to have exactly 50 targets

Now all the regulators in the network have exactly 50 transcriptional targets.

In [ ]:
n_MYC = len(network_pruned.get_reg('MYC')) # number of MYC targets in the network
EGFR = len(network_pruned.get_reg('EGFR')) # number of SERPINA12 targets in the network

print("Number of MYC targets: " + str(n_MYC) + "\nNumber of EGFR targets: " + str(EGFR))

<div class="alert alert-block alert-success">
<b></b> Run `viper` to compute the protein activity matrix (aREA method).
</div>
 
The mandatory inputs to `viper` are a gene expression signature and a gene regulatory network. We will set the output output to be an ndarray (The default output would be an AnnData object).

In [ ]:
ProtAct_aREA = pyther.viper(gex_data=gene_expr_signature, # gene expression signature
                             interactome=network_pruned, # gene regulatory network
                             enrichment = "area",
                             output_as_anndata=False,
                             njobs=1,
                             verbose=False)

`ProtAct_aREA` contains the activity of each regulatory protein computed as a Normalized Enrichment Score (NES) for each single cell.

In [ ]:
ProtAct_aREA # display the protein activity matrix 

#### Method 2 - protein activity inference using (matrix)-NaRnEA

<div class="alert alert-block alert-success">
<b></b> Run `viper` to compute the protein activity matrix (matrix-NaRnEA method).
</div>
 

Compute the protein activity using `viper` with NaRnEA as the enrichment method. Unlike aREA, NaRnEA can run with regulons of different sizes, i.e. different number of targets per regulator. Store the output in an `AnnData` object (default).

In [ ]:
ProtAct_NaRnEA = pyther.viper(gex_data=gene_expr_signature, # gene expression signature
                                 interactome=network_interactome, # gene regulatory network
                                 enrichment = "narnea",
                                 njobs=1,
                                 verbose=False)

Show the output type and length of `ProtAct_NaRnEA`.

In [ ]:
ProtAct_NaRnEA # display the protein activity matrix as AnnData object 

When using NaRnEA, `viper` returns an additional layer, `pes` that stores the Proportional Enrichment Scores (PES) for regulator, a measure bound in the interval $-1 \le PES \le 1$ that can be used as a measure of effect size. For further details, see [Griffin et al., 2022](https://pubmed.ncbi.nlm.nih.gov/36981431/).

Display NES protein activity matrix.


In [ ]:
ProtAct_NaRnEA.to_df() # NES matrix

Display the PES protein activity matrix.

In [ ]:
ProtAct_NaRnEA.to_df(layer="pes") # PES matrix

Notice that when setting the `viper` output to be an object of class `AnnData`, the input gene expression `AnnData` is stored under the attribute `gex_data`:

In [ ]:
ProtAct_NaRnEA.uns["gex_data"][0:5,0:5].to_df()

<a id='3'></a>
### Step 3. Analyze single-cells at the Protein Activity level 
We present some potential analyses and postprocessing that can be done using the protein activity matrix. 

Set the set of regulators as those for human. This is step is redundant in this case, because the default species in PyVIPER is `"human"`, but we show it to demonstrate how to have full control on the analysis.  

In [ ]:
pyther.config.set_regulators_species_to_use(species="human")

Run PCA to reduce the dimensionality of the dataset, using by projecting the PES of transcriptional regulators (TFs) and co-transcriptional regulators (coTFs) by setting `filter_by_feature_groups=["tfs", "cotfs"]`. In this example, we focus on TFs and coTFs because we are interested in proteins involved in transcriptional regulation and that are mechanistic determinants of cell state, as shown in [Paull et al., 2021](https://www.sciencedirect.com/science/article/pii/S0092867420316172?via%3Dihub) and [Califano & Alvarez, 2016](https://www.nature.com/articles/nrc.2016.124). Possible options for this parameter are `["tfs", "cotfs", "sig", "surface"]`, where the latter refer to signaling proteins and surface markers, respectively, and `None` to use all features.

In [ ]:
pyther.tl.pca(ProtAct_NaRnEA, layer="pes", filter_by_feature_groups=["tfs","cotfs"], zero_center=True,  svd_solver='arpack')

The `tl` module provides wrappers to several `scanpy.tl` functions to simplify the postprocessing of protein activity data. For instance, in the cell above we can easily subset the class of regulatory proteins to analyze. 

Compute the neighbors graph of cells using the PCA representation of the protein activity matrix. For sake of simplicity, we set 10 nearest neightbors and 50 principal components. Typically, these parameters need tuning.

In [ ]:
sc.pp.neighbors(ProtAct_NaRnEA, metric="correlation", n_neighbors=20, n_pcs=50)

Cluster at protein activity using the Leiden algorithm with `resolution=0.2` (in real applications this parameter should be optimized).

In [ ]:
sc.tl.leiden(ProtAct_NaRnEA, resolution=0.1, n_iterations=-1)

Generate a [UMAP](https://arxiv.org/abs/1802.03426) embedding of the previously computed graph. 

In [ ]:
sc.tl.umap(ProtAct_NaRnEA)

Display the 2-dimensional UMAP embedding with the corresponding function from the `pyther.pl`, a module that provides wrappers for `scanpy.pl` functions for enhanced visualization of key protein activity features. Color by cluster.  

In [ ]:
sc.set_figure_params(dpi=90)
n_cells = ProtAct_NaRnEA.shape[0]
dot_size = 120000 / n_cells*2
pyther.pl.umap(ProtAct_NaRnEA, color='leiden', palette="Set1", size=dot_size,add_outline=True)

Find the most active regulators in each cluster by integrating the NES of each regulatory protein across all cells in a cluster using the Stouffer's method: $$ NES^C_j = \frac{\sum_{k \in C}NES_{k,j}}{\sqrt{n^C}} $$ where $NES^C_j$ denotes the integrated NES for regulator $j$ in cluster/cell type $C$, $NES_{k,j}$ is the NES of regulator $j$ in cluster/cell type $C$ and $n^C$ is the number of cells in cluster $C$.

In [ ]:
ProtAct_NaRnEA = pyther.tl.stouffer(ProtAct_NaRnEA, "leiden", filter_by_feature_groups=["tfs","cotfs"]) # Stouffer integration 

Display the integrated NES scores in each cluster, that are stored in `.uns` under`stouffer`.

In [ ]:
NES_integrated = ProtAct_NaRnEA.uns["stouffer"] 
NES_integrated

Analogously, `stouffer_clusters_df` can be used to performed the same operations in Pandas DataFrames. Remove ribosomal proteins from the Stouffer-integrated matrix (ribosomal protein genes are among the most highly expressed genes in most cell types).

In [ ]:
filtered_proteins = NES_integrated.columns[~NES_integrated.columns.str.startswith('RP')].to_list() # Exclude columns starting with 'RP' from integrated matrix
NES_integrated = NES_integrated.loc[:,filtered_proteins] # remove ribosomal proteins from DataFrame

Extract the 10 most activated regulatory protein in each cluster

In [ ]:
active_proteins = NES_integrated.apply(lambda row: row.nlargest(10).index.tolist(), axis=1)  # the top 10 most activated proteins in each cell type
active_proteins

Show the NES of the top activated proteins from `ProtAct_NaRnEA` on a heatmap

In [ ]:
protein_set = active_proteins.sum() # protein set to display on the heatmap
ax = pyther.pl.heatmap(ProtAct_NaRnEA, var_names=protein_set, groupby="leiden", vcenter=0, cmap="RdBu_r",dendrogram=False,swap_axes=True, show_gene_labels=True)

### Part with literature markers
<code style="background:lightgreen">Part with literature markers (also fix title). I think to remove this part </code>

In [ ]:
markers_fibroblasts = {
    'iCAF': ['C3', 'C4B', 'GSN', 'TNXB'],
    'myCAF': ['ARHGDIB', 'CAV1', 'CD74', 'CLU', 'CXADR', 'EZR', 'HSPB1', 'NKAIN4', 'SLC9A3R1'],
    'apCAF': ['CXCL14', 'SDC1', 'TGFB1', 'THY1']
}

iCAFs_df = pd.DataFrame(columns=["regulator","target","mor","likelihood"])

# iCAFs interactome
iCAFs_df["target"] = markers_fibroblasts["iCAF"]
iCAFs_df["regulator"] = "iCAF"; iCAFs_df["mor"] = 1; iCAFs_df["likelihood"] = 1

# myCAFs interactome
myCAFs_df = pd.DataFrame(columns=["regulator","target","mor","likelihood"])
myCAFs_df["target"] = markers_fibroblasts["myCAF"]
myCAFs_df["regulator"] = "myCAF"; myCAFs_df["mor"] = 1; myCAFs_df["likelihood"] = 1

# apCAFs interactome
apCAFs_df = pd.DataFrame(columns=["regulator","target","mor","likelihood"])
apCAFs_df["target"] = markers_fibroblasts["apCAF"]
apCAFs_df["regulator"] = "apCAF"; apCAFs_df["mor"] = 1; apCAFs_df["likelihood"] = 1

CAFs_markers = pd.concat([iCAFs_df, myCAFs_df, apCAFs_df], ignore_index=True)

CAFs_interactome = pyther.Interactome("CAFs_markers", CAFs_markers)



In [ ]:
markers_enrichment = pyther.viper(ProtAct_NaRnEA, interactome=CAFs_interactome, enrichment="narnea",output_as_anndata=False, min_targets=1)

markers_enrichment = pyther.tl.nes_to_pval_df(markers_enrichment["nes"])

markers_enrichment_pval = pyther.tl.nes_to_pval_df(markers_enrichment["nes"])

np.sum(markers_enrichment_pval < 0.05)

<a id='4'></a>
### Key Takeaways
<code style="background:lightgreen">Add recap and key takeaways</code>

In [14]:
import os 
os.chdir("/Users/lucazanella7/Desktop/data/")
os.getcwd()
!curl -O https://zenodo.org/records/10056139/files/B-cell-net.tsv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  196M  100  196M    0     0  5081k      0  0:00:39  0:00:39 --:--:-- 5456k0  0:00:40  0:00:19  0:00:21 4595k


In [16]:
!open .